In [163]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import sklearn
import random
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [164]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [165]:
filename = 'Dataset/Sensor_Discrimination_normalized.csv'
df=pd.read_csv(filename)
# shuffle the DataFrame rows
df = df.sample(frac = 1) #frac = fraction
dataset=df.iloc[:,:-1]
label=df.iloc[:,-1]
print(dataset)
print(label)

            F2        F3        F1       F11        F4        F6       F10  \
576   0.259465  0.791218  0.653281  0.179255  0.526193  0.236559  0.219315   
400   0.669519  0.549293  0.258962  0.184752  0.358889  0.039582  0.209149   
1171  0.152749  0.933378  0.932419  0.349468  0.976218  0.648708  0.962726   
2032  0.206289  0.897376  0.781978  0.241844  0.836853  0.459043  0.362764   
51    0.311872  0.976783  0.865622  0.957979  0.978879  0.557724  0.961220   
...        ...       ...       ...       ...       ...       ...       ...   
1295  0.761711  0.865579  0.819785  0.195922  0.839681  0.369434  0.283509   
28    0.511230  0.531292  0.356317  0.182624  0.417595  0.037246  0.211785   
1578  0.387807  0.908984  0.942409  0.954078  0.986197  0.850192  0.971197   
528   0.783316  0.712483  0.418022  0.176401  0.533677  0.073832  0.178332   
1593  0.378824  0.975437  0.919882  0.951241  0.990687  0.824079  0.961220   

            F9        F5      F7     F12       F8  
576   0.196

In [166]:
dataset_arr=dataset.to_numpy()
# from NecessaryModules.splitData import split_data
X_train,X_comb, y_train,y_comb = train_test_split(dataset_arr, label, test_size=0.4, random_state=42)
print(len(X_train))

1327


In [167]:
X_valid, X_test, y_valid, y_test = train_test_split(X_comb, y_comb, test_size=0.50, random_state=42)
print(len(X_valid))

442


In [168]:
X_train_arr=np.array(X_train)
X_test_arr=np.array(X_test)
X_valid_arr=np.array(X_valid)

y_train_arr=np.array(y_train)
y_valid_arr=np.array(y_valid)
y_test_arr=np.array(y_test)

In [169]:
### Define the encoder dimension
encoding_dim = 3

In [170]:
# define the number of features
ncol = X_train_arr.shape[1]
ncol

12

In [171]:
from keras.layers import Input, Dense
from keras.models import Model

input_dim = Input(shape = (ncol, ))

# Encoder Layers
encoded1 = Dense(9, activation = 'relu')(input_dim)
# encoded2 = Dense(6, activation = 'relu')(encoded1)
encoded3 = Dense(encoding_dim, activation = 'relu')(encoded1)
# Decoder Layers
decoded1 = Dense(9, activation = 'relu')(encoded3)
# decoded2 = Dense(10, activation = 'relu')(decoded1)
decoded3 = Dense(ncol, activation = 'sigmoid')(decoded1)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded3)

# Compile the Model
autoencoder.compile(optimizer = 'Adam', loss = 'categorical_crossentropy')

In [172]:
autoencoder.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 12)]              0         
                                                                 
 dense_36 (Dense)            (None, 9)                 117       
                                                                 
 dense_37 (Dense)            (None, 3)                 30        
                                                                 
 dense_38 (Dense)            (None, 9)                 36        
                                                                 
 dense_39 (Dense)            (None, 12)                120       
                                                                 
Total params: 303 (1.18 KB)
Trainable params: 303 (1.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [173]:
X_train_arr.shape

(1327, 12)

In [174]:
# Train Auto Encoder
autoencoder.fit(X_train_arr, X_train_arr, epochs = 10, batch_size = 32, 
                shuffle = False, validation_data = (X_test_arr, X_test_arr))

Epoch 1/10


42/42 [==============================] - 1s 5ms/step - loss: 23.8678 - val_loss: 23.5177
Epoch 2/10
42/42 [==============================] - 0s 2ms/step - loss: 24.5716 - val_loss: 25.2484
Epoch 3/10
42/42 [==============================] - 0s 2ms/step - loss: 28.9737 - val_loss: 33.3721
Epoch 4/10
42/42 [==============================] - 0s 2ms/step - loss: 43.6262 - val_loss: 54.9337
Epoch 5/10
42/42 [==============================] - 0s 3ms/step - loss: 74.6036 - val_loss: 93.7699
Epoch 6/10
42/42 [==============================] - 0s 2ms/step - loss: 123.9912 - val_loss: 151.6248
Epoch 7/10
42/42 [==============================] - 0s 2ms/step - loss: 195.1378 - val_loss: 232.1365
Epoch 8/10
42/42 [==============================] - 0s 2ms/step - loss: 291.4268 - val_loss: 338.9472
Epoch 9/10
42/42 [==============================] - 0s 2ms/step - loss: 416.8665 - val_loss: 476.2508
Epoch 10/10
42/42 [==============================] - 0s 3ms/step - loss: 575.8787 - val_loss: 648.1087


In [175]:
encoded_train = pd.DataFrame(autoencoder.predict(X_train))

42/42 [==============================] - 0s 699us/step


In [176]:
encoded_train

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1322,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1323,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1324,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1325,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Encoder

In [177]:
# Use Encoder level to reduce dimension of train and test data

encoder = Model(inputs = input_dim, outputs = encoded3)



In [178]:
# Predict the new train and test data using Encoder

encoded_train = pd.DataFrame(encoder.predict(X_train))
encoded_train = encoded_train.add_prefix('feature_') # column level are prefix

encoded_test = pd.DataFrame(encoder.predict(X_test))
encoded_test = encoded_test.add_prefix('feature_')

 1/42 [..............................] - ETA: 2s

14/14 [==============================] - 0s 1ms/step


In [179]:
encoded_X = pd.DataFrame(encoder.predict(dataset))
encoded_X = encoded_X.add_prefix('feature_')

70/70 [==============================] - 0s 1ms/step


In [180]:
encoded_train

,feature_0,feature_1,feature_2
0,77.570183,99.477859,0.0
1,42.126438,53.634796,0.0
2,36.110622,45.955551,0.0
3,49.041031,62.809341,0.0
4,43.123039,55.576515,0.0
...,...,...,...
1322,48.208313,62.130074,0.0
1323,73.745361,94.634163,0.0
1324,41.879974,53.368671,0.0
1325,48.515182,62.477444,0.0


In [185]:
import pickle
pickle.dump(encoder,open("all_pkl_file/Encoder_3.pkl","wb"))

# MOdel try

In [182]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler(copy=True).fit(encoded_train.values)
encoded_train_n=scaler.transform(encoded_train)
encoded_train_n

array([[0.57175404, 0.57239467, 0.        ],
       [0.3105054 , 0.30861413, 0.        ],
       [0.26616406, 0.2644278 , 0.        ],
       ...,
       [0.30868876, 0.30708286, 0.        ],
       [0.35759553, 0.35949466, 0.        ],
       [0.30376938, 0.30540732, 0.        ]], dtype=float32)

In [183]:
# train the model with random parameter
from sklearn.svm import SVC

model_SVC = SVC(
        C=8.75,#Regularization parameter
        kernel="rbf", # maximum number of estimators at which boosting is terminated
        gamma=0.4,
        tol=0.483,
        decision_function_shape='ovo',
        
     )

model_SVC.fit(encoded_train, y_train)

SVC(C=8.75, decision_function_shape='ovo', gamma=0.4, tol=0.483)

In [184]:
# Finding mean absolute error
preds = model_SVC.predict(encoded_test)
preds = preds.astype(int)
mae = np.abs(y_test-preds).mean()
mae

0.17607223476297967